# Introduction
train and test data are measured with same roll resistance (lever position 3) and wheel pressure.


# Analyze
Question: Which features and model is good enough to be predict virtual power?

_Train data (app data):
* X of app data set -> cadence
* Y -> velocity

## Pre processing


In [11]:
import pandas as pd

from src.tcx import read_tcx, Tcx, COLUMN_NAME_SPEED, COLUMN_NAME_WATTS
from src.test_data import TrainDataSet

tcx_app: Tcx = read_tcx('test/tcx/cadence_1612535177298-gear7.tcx')
tcx_tacx: Tcx = read_tcx('test/tcx/tacx-activity_6225123072-gear7-resistance3.tcx')
tcx_tacx_gear20: Tcx = read_tcx('test/tcx/tacx-activity_6225123072-gear7-resistance3.tcx')

# generate test data
dts_tacx: TrainDataSet = TrainDataSet(tcx_tacx)

def print_stats(df: pd.DataFrame):

    #print(df[['Speed', 'Cadence', 'Ext.Watts', 'DistanceMeters', 'DistanceMeters-delta', 'Ext.Speed', 'Time', 'Time-delta']].head(100))

    # print(df[[COLUMN_NAME_SPEED]].describe())
    print(df
          .assign(speed_int=lambda x: x[COLUMN_NAME_SPEED].apply(lambda x : int(x)))
          .groupby(by=['speed_int'])[COLUMN_NAME_WATTS].describe())

print_stats(dts_tacx.get_dataframe())

           count        mean        std    min     25%    50%     75%    max
speed_int                                                                   
22           2.0  100.500000  21.920310   85.0   92.75  100.5  108.25  116.0
23           8.0   85.000000   2.878492   81.0   84.00   85.0   86.50   89.0
24          89.0   86.325843   5.961573   72.0   84.00   86.0   89.00  100.0
25          44.0   98.250000   9.983999   72.0   98.00  100.0  105.25  112.0
26          83.0  103.180723   9.753229   82.0  100.00  102.0  112.00  124.0
27         109.0  115.798165   7.678469   81.0  112.00  116.0  120.00  139.0
28          63.0  126.571429   7.572689  105.0  122.00  124.0  132.00  148.0
29          39.0  138.666667  12.103356  122.0  124.50  142.0  149.50  153.0
30          10.0  145.600000   9.070097  124.0  146.00  149.5  150.75  152.0


## Linear regression
To find out we create a linear model from the tacx data with differetn features and calculate a score for each tacx cadence value to see whether the model predicts the velocity good enough.

### Cadence
_X_ = [cadence], _y_ = power

In [10]:
from math import sqrt

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import max_error, mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=7)

# train
alg: LinearRegression = LinearRegression().fit(X_train, y_train)

# test
X_train_np = X_train.to_numpy()
y_train_np = y_train.to_numpy()
y_predicted = alg.predict(X_test)

print("Shape X_train/X_test: {}/{}".format(X_train.shape, X_test.shape))
# Score on X_test
print("Error R²: {:.2f}".format(alg.score(X_test, y_test)))
mse = mean_squared_error(y_test.to_numpy(), y_predicted)
print("MSE error (mean squared error / variance): {:.2f}".format(mse))
print("sqrt(MSE) (standard deviation): {:.2f}".format(sqrt(mse)))
print("Max error: {}".format(max_error(y_test, y_predicted)))

# cross k-fold validation (k=5)
scores: list = cross_val_score(alg, X_test.to_numpy(), y_test.to_numpy(), cv=5)
print("Cross validation: {}".format(scores))

# max error y_test <-> y_predict

NameError: name 'X' is not defined

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Plot X_train -> y_train
plt.scatter(X_train, y_train,  color='black')
plt.plot(X_train, alg.predict(X_train), color='blue', linewidth=2)

plt.xticks()
plt.yticks()
plt.show()

# Plot histogram of absolute error of |y_test - y_predicted|
plt.hist(y_test - y_predicted, 40, density=True, facecolor='g', alpha=0.75)
plt.show()


